In [ ]:
%region us-east-1
%iam_role #setar
%idle_timeout 15
%glue_version 4.0
%worker_type G.1X
%number_of_workers 5

#%additional_python_modules s3://fiocruz-datalake-bucket/raw/aesop/dados_auxiliares/python_utils/teste.py
%extra_py_files s3://fiocruz-datalake-bucket/raw/aesop/dados_auxiliares/python_utils/util_custom.py

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

In [ ]:
from util_custom import *

In [ ]:
#%stop_session

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
import os

In [ ]:
print("bases anuais {}".format(aesop_csv_anual_path))
print("bases anuais abp {}".format(aesop_csv_anual_ABP_path))
print("bases anuais 2023 ate 01-10-2023 {}".format(aesop_csv_2023_path))
print("bases anuais de 02-10-2023 em diante {}".format(aesop_csv_2023_atual_path))

#### leitura

In [ ]:
schema = StructType(
   [StructField('ANO', IntegerType(), True),
    StructField('SEMANA', IntegerType(), True),
    StructField('SG_UF', StringType(), True),
    StructField('CO_MUNICIPIO_IBGE', StringType(), True),
    StructField('SEXO', StringType(), True),
    StructField('FX_ETARIA', StringType(), True),
    StructField('CIDCIAP', StringType(), True),
    StructField('QT', IntegerType(), True)
   ]
  )

df_anual = spark.read.options(header='True', delimiter=',')\
    .schema(schema)\
.csv(aesop_csv_anual_path)

df_anual_abp = spark.read.options(header='True', delimiter=',')\
    .schema(schema)\
.csv(aesop_csv_anual_ABP_path)

df_2023_ate01102023 = spark.read.options(header='True', delimiter=',')\
    .schema(schema)\
.csv(aesop_csv_2023_path)

df_2023_atual = spark.read.options(header='True', delimiter=',')\
    .schema(schema)\
.csv(aesop_csv_2023_atual_path)



In [ ]:
#print("df anual {}".format(df_anual.count()))
#print("df anual abp {}".format(df_anual_abp.count()))
#print("df 2023 ate 01102023 abp {}".format(df_2023_ate01102023.count()))
#print("df 02-10-2023 ate atual {}".format(df_2023_atual.count()))

In [ ]:
df = df_anual.unionByName(df_anual_abp)
df = df.unionByName(df_2023_ate01102023)
df = df.unionByName(df_2023_atual)

In [ ]:
#df.count() # 1208951517

In [ ]:
len(df.columns)

#### criacao data folder

In [ ]:
df = df.withColumn("INPUT_FILE", F.input_file_name())\
.withColumn('Data_folder', F.regexp_extract('INPUT_FILE', '([0-9]{8})', 1))\
.withColumn("Data_folder",F.to_date(F.col("Data_folder"),"yyyyMMdd"))\
.drop('INPUT_FILE')#.checkpoint()

In [ ]:
df.groupBy(F.col('Data_folder')).count().orderBy(F.desc('Data_folder')).show()

#### cast ano e semana

In [ ]:
df = df.withColumn("ANO", F.col("ANO").cast(ShortType()))\
.withColumn("SEMANA", F.col("SEMANA").cast(ByteType()))

In [ ]:
#get_parquet_file_name() #'aesop_dados_2017_20240310.parquet'

#### escrita

In [ ]:
df.write.parquet("s3://fiocruz-datalake-bucket/raw/aesop/dados-ms-parquet/aesop_dados_2017_20240414.parquet", mode='overwrite')

In [ ]:
df_check = spark.read.parquet("s3://fiocruz-datalake-bucket/raw/aesop/dados-ms-parquet/aesop_dados_2017_20240414.parquet")

In [ ]:
df_check.count() #1091155616 ->1095193151 ->1096239903 ->1099462094 ->1100855294

In [ ]:
#df.write.parquet(aesop_parquet_path+get_parquet_file_name(), mode='overwrite')
#aesop_parquet_path+get_parquet_file_name()

In [ ]:
%stop_session